# Custom Chatbot Project

TODO: The option that I like the most is NYC Food Scrap Drop-off Sites, I like the nature of the challenge and the location, besides it fulfills the requirements of the project.



## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [69]:
import pandas as pd

data_path = "data/nyc_food_scrap_drop_off_sites.csv"
data = pd.read_csv(data_path)
print(data.shape)
data.head()

(479, 19)


,Borough,NTAName,SiteName,SiteAddr,Hosted_By,Open_Month,Day_Hours,Notes,Website,BoroCD,CouncilDis,ct2010,BBL,BIN,Latitude,Longitude,PolicePrec,Object ID,Location Point
0,Manhattan,Inwood,SW West 212th Street & 10th Avenue,NaN,Department of Sanitation,Year Round,24/7,Download the app to access bins. Accepts all f...,www.nyc.gov/smartcomposting,112,10,NaN,NaN,NaN,NaN,NaN,34,6914,NaN
1,Brooklyn,Park Slope,Old Stone House Brooklyn,"336 3rd St, Brooklyn, NY 11215",Old Stone House Brooklyn,Year Round,24/7 (Start Time: 24/7 - End Time: 24/7),NaN,NaN,306,39,NaN,NaN,NaN,40.672712,-73.984731,78,6777,POINT (-73.984731 40.6727118)
2,Bronx,Norwood,NE Hull Avenue & East 207th Street,NaN,Department of Sanitation,Year Round,24/7,Download the app to access bins. Accepts all f...,www.nyc.gov/smartcomposting,207,11,NaN,NaN,NaN,NaN,NaN,52,7115,NaN
3,Manhattan,East Harlem (South),SE East 112th Street & Park Avenue,NaN,Department of Sanitation,Year Round,24/7,Download the app to access bins. Accepts all f...,www.nyc.gov/smartcomposting,111,8,NaN,NaN,NaN,NaN,NaN,23,6819,NaN
4,Queens,Astoria (Central),Astoria Pug: Broadway,"32-11 31st St, Queens, NY 11106",Astoria Pug,Year Round,Saturdays (Start Time: 9:00 AM - End Time: 3:...,"Not accepted: meat, bones, or dairy",https://www.instagram.com/astoriapug/?hl=en,401,22,NaN,NaN,NaN,40.761472,-73.925611,114,7046,POINT (-73.925611 40.761472)


In [70]:
# Combine relevant columns into a single "text" column
data['text'] = "Borough: " + data['Borough'] + ", " + "Site Name: " + data['SiteName'] + ", " + "Address: " + data['SiteAddr'] + ", " + "Hosted By: " + data['Hosted_By'] + ", " + "Open Months: " + data['Open_Month'] + ", " + "Days and Hours: " + data['Day_Hours'] + ", " + "Notes: " + data['Notes'] + ", " + "Website: " + data['Website']

non_empty_text_mask = data["text"].str.len() > 0
data = data[non_empty_text_mask]

print(data.shape)

(197, 20)


In [71]:
data = data[['text']]
print(data.shape)
data.head()

(197, 1)


,text
4,"Borough: Queens, Site Name: Astoria Pug: Broad..."
5,"Borough: Brooklyn, Site Name: East New York Fa..."
9,"Borough: Manhattan, Site Name: Battery Park Ci..."
12,"Borough: Bronx, Site Name: Drew Gardens, Addre..."
20,"Borough: Staten Island, Site Name: *CLOSED FOR..."


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.


# Generate embeddings for dataset:

In [72]:
import os
import openai
from openai import OpenAI

from constants import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)


In [73]:
embeddings_model_name = "text-embedding-3-small"

response = client.embeddings.create(
    input = data["text"].tolist(), 
    model=embeddings_model_name
)

In [74]:
print(type(response))
print(len(response.data))
print(response.data[0])

<class 'openai.types.create_embedding_response.CreateEmbeddingResponse'>
197
Embedding(embedding=[-0.037016306072473526, -0.006045001558959484, 0.003671778831630945, 0.022460538893938065, -0.003325497265905142, -0.0381387360394001, -0.08048061281442642, -0.01049591414630413, 0.011719840578734875, 0.003919549286365509, 0.02304563671350479, -0.02550542913377285, 0.014090077951550484, 0.01368409302085638, -0.018269337713718414, 0.07279077917337418, 0.005221090279519558, 0.010119780898094177, 0.0032687787897884846, 0.019451472908258438, 0.03878353536128998, -0.005812156945466995, -0.047524161636829376, -0.04465838149189949, 0.02358296886086464, 0.0431060865521431, -0.01152878813445568, -0.0027150267269462347, 0.0713101252913475, 0.003922534640878439, -0.04919586703181267, -0.02946975640952587, 0.01885443553328514, 0.02323668822646141, 0.012525840662419796, -0.011266091838479042, -0.008030150085687637, -0.00958841759711504, 0.002434419235214591, 0.05091533437371254, -0.024932272732257843, -

In [75]:
length_response = len(response.data)
data["embeddings"] = [response.data[i].embedding for i in range(length_response)]
data.head()

,text,embeddings
4,"Borough: Queens, Site Name: Astoria Pug: Broad...","[-0.037016306072473526, -0.006045001558959484,..."
5,"Borough: Brooklyn, Site Name: East New York Fa...","[-0.032146044075489044, -0.0037656286731362343..."
9,"Borough: Manhattan, Site Name: Battery Park Ci...","[-0.010767555795609951, -0.02981601469218731, ..."
12,"Borough: Bronx, Site Name: Drew Gardens, Addre...","[-0.03569330647587776, -0.0030808832962065935,..."
20,"Borough: Staten Island, Site Name: *CLOSED FOR...","[-0.04941139742732048, -0.019610701128840446, ..."


In [76]:
data.to_csv("embeddings.csv")

In [77]:
!ls 

__pycache__    constants.py   data           embeddings.csv project.ipynb


In [78]:
import numpy as np

data = pd.read_csv("embeddings.csv", index_col=0)
data["embeddings"] = data["embeddings"].apply(eval).apply(np.array)
data.head()


,text,embeddings
4,"Borough: Queens, Site Name: Astoria Pug: Broad...","[-0.037016306072473526, -0.006045001558959484,..."
5,"Borough: Brooklyn, Site Name: East New York Fa...","[-0.032146044075489044, -0.0037656286731362343..."
9,"Borough: Manhattan, Site Name: Battery Park Ci...","[-0.010767555795609951, -0.02981601469218731, ..."
12,"Borough: Bronx, Site Name: Drew Gardens, Addre...","[-0.03569330647587776, -0.0030808832962065935,..."
20,"Borough: Staten Island, Site Name: *CLOSED FOR...","[-0.04941139742732048, -0.019610701128840446, ..."


# Generate embeddings for question:

In [83]:
from scipy.spatial.distance import cosine


def generate_question_embeddings(input_question, dataframe):

    input_data = dataframe.copy()
    
    embedding_question = client.embeddings.create(
        input = input_question, 
        model=embeddings_model_name
    ).data[0].embedding

    embedding_question_array = np.array(embedding_question)

    input_data["distances"] = input_data["embeddings"].apply(
        lambda x: cosine(embedding_question_array, x)
    )

    input_data.sort_values(by="distances", ascending=True, inplace=True)

    return input_data

sample_question = """
Question: "What is the Name for this site: Newkirk Ave & Nostrand Ave"
Answer:
"""   

df = generate_question_embeddings(
    input_question = sample_question, 
    dataframe = data
)
df.head()


,text,embeddings,distances
54,"Borough: Brooklyn, Site Name: Little Haiti Foo...","[-0.017932480201125145, -0.01911027356982231, ...",0.509549
216,"Borough: Brooklyn, Site Name: Crown Heights, A...","[-0.0203903466463089, -0.013577821664512157, 0...",0.524534
204,"Borough: Brooklyn, Site Name: Avenue X at West...","[-0.01457306370139122, -0.020756671205163002, ...",0.536922
445,"Borough: Brooklyn, Site Name: Bay Parkway at 8...","[-0.010738720186054707, -0.007316163275390863,...",0.545379
125,"Borough: Brooklyn, Site Name: North Brooklyn C...","[-0.02690531685948372, -0.0025081734638661146,...",0.547211


In [84]:
import tiktoken

def create_prompt(question, df, max_token_count):

    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    num_tokens_template = len(tokenizer.encode(prompt_template))
    num_tokens_question = len(tokenizer.encode(question))
    current_token_count =  num_tokens_template + num_tokens_question
                            
    context = []
    for text in generate_question_embeddings(question, df)["text"].values:
        
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [86]:
sample_prompt = create_prompt(sample_question, data, max_token_count=750)
print(sample_prompt)


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

Borough: Brooklyn, Site Name: Little Haiti Food Scrap Drop-off Site, Address: Newkirk Ave & Nostrand Ave, Hosted By: GrowNYC, Open Months: Year Round, Days and Hours: Wednesdays (Start Time: 9:30 AM - End Time:  1:30 PM), Notes: Not accepted: meat, bones, or dairy, Website: grownyc.org/compost

###

Borough: Brooklyn, Site Name: Crown Heights, Address: Nostrand Ave & Crown St, Hosted By: GrowNYC, Open Months: Year Round, Days and Hours: Fridays (Start Time: 9:00 AM - End Time:  2:00 PM), Notes: Not accepted: meat, bones, or dairy, Website: grownyc.org/compost

###

Borough: Brooklyn, Site Name: Avenue X at West Street, Address: SW corner of Avenue X and West Street, in front of Rite Aide, across from P.S.216, Hosted By: NYC Compost Project Hosted by LES Ecology Center, Open Months: Year Round, Days and Hours: Wednesdays (Start Time: 10:00 AM - End 

In [100]:
def custom_query(
        input_question, 
        dataframe, 
        max_prompt_tokens=3000, 
        max_answer_tokens=750):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(
        input_question, 
        dataframe, 
        max_prompt_tokens
    )



    try:

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            max_tokens=max_answer_tokens
        )
        return response.choices[0].message.content
    
    except Exception as e:
        print(e)
        return ""
    


In [101]:
custom_query(
        sample_prompt, 
        data, 
)

'The site name for Newkirk Ave & Nostrand Ave is "Little Haiti Food Scrap Drop-off Site"'

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [108]:
first_prompt = "When is open Battery Park City Authority Rockefeller Park?"

In [109]:
custom_query(
        first_prompt, 
        data, 
)

'It is open year-round, 24/7.'

In [107]:
data["text"].iloc[2]

'Borough: Manhattan, Site Name: Battery Park City Authority Rockefeller Park, Address: Lower level downstairs at corner of Chambers St and River Terrace, Hosted By: Battery Park City Authority, Open Months: Year Round, Days and Hours: 24/7 (Start Time: 24/7 - End Time:  24/7), Notes: Not accepted: meat, bones, or dairy, Website: https://bpca.ny.gov/nature-and-sustainability/sustainability/'

### Question 2

In [105]:
second_prompt = "What is the address of Astoria Pug?"

In [106]:
custom_query(
        second_prompt, 
        data, 
)

'Astoria Pug has multiple locations with different addresses, such as Newtown (30-36 41st Street), Broadway (32-11 31st St), Most Precious Blood (32-23 36th St), 41st Street (Ditmars Boulevard and 41st Street), Steinway (38-12 30th Ave), and Hoyt (Northwest corner of Hoyt Avenue North and 21st Street).'

In [104]:
data.text.iloc[0]

'Borough: Queens, Site Name: Astoria Pug: Broadway, Address: 32-11 31st St, Queens, NY 11106, Hosted By: Astoria Pug, Open Months: Year Round, Days and Hours: Saturdays (Start Time: 9:00 AM - End Time:  3:00 PM), Notes: Not accepted: meat, bones, or dairy, Website: https://www.instagram.com/astoriapug/?hl=en'